In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.optimizers.schedules import ExponentialDecay
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rhrua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rhrua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

with open('Dataset\intent.json') as file:
  data = json.load(file)

words = []
groups = []
labels = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    
    word = word_tokenize(pattern)
    words.extend(word)
    groups.append((word,intent['tag']))

    if intent['tag'] not in labels:
        labels.append(intent['tag'])


words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() ]
words = sorted(list(set(words)))

labels = sorted(list(set(labels)))


training = []
output = [0] * len(labels)

for group in groups:
    bag = []
    bag_check = group[0]
    checked = []
    for word in bag_check:
      checked_word = lemmatizer.lemmatize(word.lower())
      checked.append(checked_word)
    bag_check = checked
   
    for w in words:
      if w in bag_check:
         bag.append(1)
      else:
         bag.append(0)
   
    output_row = list(output)
    output_row[labels.index(group[1])] = 1
   
    training.append([bag,output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])
   
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))  # Increased units
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))  # Increased layers and units
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))   # Another hidden layer
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))


initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)


model.compile(optimizer=SGD(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)





def get_input (inp):
   inpSpl = word_tokenize(inp)
   inpSpl = [lemmatizer.lemmatize(word.lower()) for word in inpSpl if word.isalnum()]
   return inpSpl


def bag_input(input, words):
   sorted_input = get_input(input)
   store = [0] * len(words)
   for word in sorted_input:
      for w in words:
         if word == w:
            store[words.index(w)] = 1
   return store
      

def predict_ans (input):
   pred = model.predict(np.array([bag_input(input, words)]))
   pred_index = np.argmax(pred)  
   return pred_index

def get_response(predicted_class_index):
    tag = labels[predicted_class_index]
    for intent in data['intents']:  
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            break
    return response
while True:
    user_input = input("You: ")  # Get input from the user
    if user_input.lower() in ['exit', 'quit', 'bye']:  # Exit conditions
        print("Chatbot: Goodbye!")
        break
    predicted_class_index = predict_ans(user_input)  # Predict intent
    response = get_response(predicted_class_index)  # Generate response
    print("Chatbot:", response)     

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0